# Data Loading

In [1]:
import asyncio
import aiohttp
from tqdm.auto import tqdm

In [2]:
# Downloading the new york city taxi data locally
MONTH = 13
YEAR = [2024]

# Creating a list of urls to download the data
url_list = []
for month, year in [(m, y) for m in range(1, MONTH) for y in YEAR]:
    if month < 10:
        month = f'0{month}'
    
    yellow_taxi_url = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month}.parquet'
    green_taxi_url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_{year}-{month}.parquet"
    fhv_url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_{year}-{month}.parquet"
    fhv_hv_url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_{year}-{month}.parquet"
    
    url_list.extend([yellow_taxi_url, green_taxi_url, fhv_url, fhv_hv_url])

url_list

['https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2024-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2024-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2024-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-03.parquet',
 'https

In [3]:
# Downloading the data in the nyc_taxi_data directory
data_dir = '../nyc_taxi_data'

# Establishing a connection with the server and downloading the data locally

# This functions downloads file by streaming method
async def download_file(url):
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:
                if response.status == 200:
                    # Getting file name
                    if "content-disposition" in response.headers:
                        header = response.headers["content-disposition"]
                        file_name = header.split("filename=")[1]
                    else:
                        file_name = url.split("/")[-1]
                    
                    # Getting the file size and updating the download status
                    total_size = int(response.headers.get('Content-Length', 0))
                    downloaded = 0

                    # Getting the data in chunked form and writing it to a file
                    with open(f'{file_name}', 'wb') as f:
                        while True:
                            chunk = await response.content.read(4096)
                            if not chunk:
                                break
                            f.write(chunk)
                            downloaded += len(chunk)
                            print(f"Downloading {file_name}: {downloaded}/{total_size} bytes", end='\r')
                        print(f'\n{data_dir}/{file_name} downloaded successfully.')
                else:
                    print(f"Connection Error, received status code: {response.status} for URL: {url}")
    except aiohttp.ClientConnectionError:
        print(f"Connection Error for URL: {url}")
    except aiohttp.ClientResponseError:
        print(f"Invalid response for URL: {url}")

async def main(url_list):
    batch_size = 5
    for i, start in enumerate(range(0, len(url_list), batch_size)):
        tasks = [download_file(url) for url in tqdm(
            url_list[start: start+batch_size], 
            desc=f'Batch {i}',
            )]
        await asyncio.gather(*tasks)
    print('All the tasks are completed.')

In [4]:
# Calling the main function in await because of ipython functionality,
# use asyncio.run(main()) outside of notebook
await main(url_list=url_list)

  0%|          | 0/5 [00:00<?, ?it/s]

../nyc_taxi_data/green_tripdata_2024-01.parquet downloaded successfully.
../nyc_taxi_data/fhv_tripdata_2024-01.parquet downloaded successfully.
../nyc_taxi_data/yellow_tripdata_2024-01.parquet downloaded successfully.
../nyc_taxi_data/yellow_tripdata_2024-02.parquet downloaded successfully.
../nyc_taxi_data/fhvhv_tripdata_2024-01.parquet downloaded successfully.


  0%|          | 0/5 [00:00<?, ?it/s]

../nyc_taxi_data/green_tripdata_2024-02.parquet downloaded successfully.
../nyc_taxi_data/green_tripdata_2024-03.parquet downloaded successfully.
../nyc_taxi_data/fhv_tripdata_2024-02.parquet downloaded successfully.
../nyc_taxi_data/yellow_tripdata_2024-03.parquet downloaded successfully.
../nyc_taxi_data/fhvhv_tripdata_2024-02.parquet downloaded successfully.


  0%|          | 0/5 [00:00<?, ?it/s]

...

../nyc_taxi_data/green_tripdata_2024-10.parquet downloaded successfully.
../nyc_taxi_data/fhv_tripdata_2024-10.parquet downloaded successfully.
../nyc_taxi_data/yellow_tripdata_2024-10.parquet downloaded successfully.
../nyc_taxi_data/fhvhv_tripdata_2024-10.parquet downloaded successfully.
../nyc_taxi_data/fhvhv_tripdata_2024-09.parquet downloaded successfully.


  0%|          | 0/5 [00:00<?, ?it/s]

../nyc_taxi_data/green_tripdata_2024-11.parquet downloaded successfully.
../nyc_taxi_data/fhv_tripdata_2024-11.parquet downloaded successfully.
../nyc_taxi_data/yellow_tripdata_2024-11.parquet downloaded successfully.
../nyc_taxi_data/yellow_tripdata_2024-12.parquet downloaded successfully.
../nyc_taxi_data/fhvhv_tripdata_2024-11.parquet downloaded successfully.


  0%|          | 0/3 [00:00<?, ?it/s]

../nyc_taxi_data/green_tripdata_2024-12.parquet downloaded successfully.
../nyc_taxi_data/fhv_tripdata_2024-12.parquet downloaded successfully.
../nyc_taxi_data/fhvhv_tripdata_2024-12.parquet downloaded successfully.


In [7]:
import pandas as pd
import pyarrow.parquet as pq

In [5]:
# Read metadata
pq.read_metadata('../nyc_taxi_data/yellow_tripdata_2024-01.parquet')

  created_by: parquet-cpp-arrow version 14.0.2
  num_columns: 19
  num_rows: 2964624
  num_row_groups: 3
  format_version: 2.6
  serialized_size: 6357

In [8]:
# Read file, read the table from file and check schema
file = pq.ParquetFile('../nyc_taxi_data/yellow_tripdata_2024-01.parquet')
table = file.read()
table.schema

VendorID: int32
tpep_pickup_datetime: timestamp[us]
tpep_dropoff_datetime: timestamp[us]
passenger_count: int64
trip_distance: double
RatecodeID: int64
store_and_fwd_flag: large_string
PULocationID: int32
DOLocationID: int32
payment_type: int64
fare_amount: double
extra: double
mta_tax: double
tip_amount: double
tolls_amount: double
improvement_surcharge: double
total_amount: double
congestion_surcharge: double
Airport_fee: double

In [7]:
df = pd.read_parquet('../nyc_taxi_data/yellow_tripdata_2024-01.parquet')
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.70,1.00,0.5,0.00,0.00,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.00,3.50,0.5,3.75,0.00,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.30,3.50,0.5,3.00,0.00,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.00,3.50,0.5,2.00,0.00,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.90,3.50,0.5,3.20,0.00,1.0,16.10,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,2024-01-31 23:45:59,2024-01-31 23:54:36,NaN,3.18,NaN,None,107,263,0,15.77,0.00,0.5,2.00,0.00,1.0,21.77,NaN,NaN
2964620,1,2024-01-31 23:13:07,2024-01-31 23:27:52,NaN,4.00,NaN,None,114,236,0,18.40,1.00,0.5,2.34,0.00,1.0,25.74,NaN,NaN
2964621,2,2024-01-31 23:19:00,2024-01-31 23:38:00,NaN,3.33,NaN,None,211,25,0,19.97,0.00,0.5,0.00,0.00,1.0,23.97,NaN,NaN
2964622,2,2024-01-31 23:07:23,2024-01-31 23:25:14,NaN,3.06,NaN,None,107,13,0,23.88,0.00,0.5,5.58,0.00,1.0,33.46,NaN,NaN


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964624 entries, 0 to 2964623
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [9]:
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL

In [10]:
# Creating an open SQL database connection object
postgres_url = URL.create(drivername='postgresql',
                          username='postgres',
                          password='postgres',
                          host='localhost',
                          port=5432,
                          database='ny_taxi',)
engine = create_engine(url=postgres_url)
engine.connect()

In [11]:
# Getting the schema from the dataframe in SQL format
print(pd.io.sql.get_schema(df, name='yellow_taxi_data', con=engine))


CREATE TABLE yellow_taxi_data (
	"VendorID" INTEGER, 
	tpep_pickup_datetime TIMESTAMP WITHOUT TIME ZONE, 
	tpep_dropoff_datetime TIMESTAMP WITHOUT TIME ZONE, 
	passenger_count FLOAT(53), 
	trip_distance FLOAT(53), 
	"RatecodeID" FLOAT(53), 
	store_and_fwd_flag TEXT, 
	"PULocationID" INTEGER, 
	"DOLocationID" INTEGER, 
	payment_type BIGINT, 
	fare_amount FLOAT(53), 
	extra FLOAT(53), 
	mta_tax FLOAT(53), 
	tip_amount FLOAT(53), 
	tolls_amount FLOAT(53), 
	improvement_surcharge FLOAT(53), 
	total_amount FLOAT(53), 
	congestion_surcharge FLOAT(53), 
	"Airport_fee" FLOAT(53)
)




In [39]:
# Inserting the data in the postgres table in batch format

# Creating batches of 100,000 from the parquet file
batch_size = 100_000
batches_iter = file.iter_batches(batch_size=batch_size)
batches_iter

# taking the first batch for testing the insertion
test_df = next(batches_iter).to_pandas()
test_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1,1.72,1,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1,1.80,1,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1,4.70,1,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1,1.40,1,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1,0.80,1,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1,2024-01-02 13:34:18,2024-01-02 13:38:13,1,0.70,1,N,263,75,1,5.8,2.5,0.5,2.00,0.0,1.0,11.80,2.5,0.0
99996,1,2024-01-02 13:47:42,2024-01-02 13:59:26,1,1.70,1,N,236,238,1,11.4,2.5,0.5,4.60,0.0,1.0,20.00,2.5,0.0
99997,2,2024-01-02 13:23:51,2024-01-02 13:36:29,1,1.87,1,N,236,162,1,12.8,0.0,0.5,3.36,0.0,1.0,20.16,2.5,0.0
99998,2,2024-01-02 13:42:04,2024-01-02 13:48:39,1,1.58,1,N,229,263,2,9.3,0.0,0.5,0.00,0.0,1.0,13.30,2.5,0.0


In [45]:
# Creating the table in postgres using the dataframe header
test_df.head(0).to_sql(name='yellow_tripdata_2024_01', con=engine, if_exists='replace')

0

In [15]:
# Inserting the test dataframe in the postgres table
test_df.to_sql(name='yellow_tripdata_2024_01', con=engine, if_exists='append')

# Querying the database to check whether the data was inserted
with engine.connect() as con:
    select_query = ("SELECT * "
                    "FROM yellow_tripdata_2024_01;")

    delete_query = ("DELETE "
                    "FROM yellow_tripdata_2024_01;")
    
    # Column names
    columns = con.execute(statement=text(text=select_query)).keys()
    print('Columns:', columns._keys)

    # First 5 rows
    result = con.execute(statement=text(text=select_query))
    for row in result.fetchmany(5):
        print(row)
    print('Total rows:', result.rowcount)

    # Deleting the test data
    result = con.execute(statement=text(text=delete_query))
    con.commit()
    print('\nAfter Deleting the test data:')
    
    columns = con.execute(statement=text(text=select_query)).keys()
    print('Columns: ', columns._keys)
    result = con.execute(statement=text(text=select_query))
    print('Total rows:', result.rowcount)

Columns: ['index', 'VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'Airport_fee']
(0, 2, datetime.datetime(2024, 1, 1, 0, 57, 55), datetime.datetime(2024, 1, 1, 1, 17, 43), 1, 1.72, 1, 'N', 186, 79, 2, 17.7, 1.0, 0.5, 0.0, 0.0, 1.0, 22.7, 2.5, 0.0)
(1, 1, datetime.datetime(2024, 1, 1, 0, 3), datetime.datetime(2024, 1, 1, 0, 9, 36), 1, 1.8, 1, 'N', 140, 236, 1, 10.0, 3.5, 0.5, 3.75, 0.0, 1.0, 18.75, 2.5, 0.0)
(2, 1, datetime.datetime(2024, 1, 1, 0, 17, 6), datetime.datetime(2024, 1, 1, 0, 35, 1), 1, 4.7, 1, 'N', 236, 79, 1, 23.3, 3.5, 0.5, 3.0, 0.0, 1.0, 31.3, 2.5, 0.0)
(3, 1, datetime.datetime(2024, 1, 1, 0, 36, 38), datetime.datetime(2024, 1, 1, 0, 44, 56), 1, 1.4, 1, 'N', 79, 211, 1, 10.0, 3.5, 0.5, 2.0, 0.0, 1.0, 17.0, 2.5, 

In [46]:
from time import perf_counter

# Inserting all the data into the table
start_time = perf_counter()

batch_size = 100_000
batches_iter = file.iter_batches(batch_size=batch_size)

data_inserted = 0
for i, batch in enumerate(iterable=batches_iter,
                          start=1,):
    batch_df = batch.to_pandas()
    print(f'Inserting Batch-{i} data in table yellow_tripdata_2024_01.')
    
    batch_start_time = perf_counter()
    batch_df.to_sql(name='yellow_tripdata_2024_01',
                    con=engine,
                    if_exists='append')
    batch_end_time = perf_counter()
    data_inserted += len(batch)
    print(f'Duration for Batch-{i} data insertion: {(batch_end_time - batch_start_time):.3f} (seconds).')
    print(f'Total data inserted: {data_inserted}\n')

end_time = perf_counter()
print(f'Completed data insertion, time taken: {(end_time - start_time):.3f} (seconds).\n')

Inserting Batch-1 data in table yellow_tripdata_2024_01.
Duration for Batch-1 data insertion: 9.991 (seconds).
Total data inserted: 100000

Inserting Batch-2 data in table yellow_tripdata_2024_01.
Duration for Batch-2 data insertion: 9.785 (seconds).
Total data inserted: 200000

Inserting Batch-3 data in table yellow_tripdata_2024_01.
Duration for Batch-3 data insertion: 9.987 (seconds).
Total data inserted: 300000

Inserting Batch-4 data in table yellow_tripdata_2024_01.
Duration for Batch-4 data insertion: 10.229 (seconds).
Total data inserted: 400000

Inserting Batch-5 data in table yellow_tripdata_2024_01.
Duration for Batch-5 data insertion: 9.910 (seconds).
Total data inserted: 500000

Inserting Batch-6 data in table yellow_tripdata_2024_01.
Duration for Batch-6 data insertion: 9.996 (seconds).
Total data inserted: 600000

Inserting Batch-7 data in table yellow_tripdata_2024_01.
Duration for Batch-7 data insertion: 9.950 (seconds).
Total data inserted: 700000

Inserting Batch-8 d